# Libraries

In [1]:
# import pandas as pd
# import numpy as np
# import time
# import twython
# import requests
# import nltk
# import warnings
# warnings.filterwarnings('ignore')

# from htmldate import find_date
# from tqdm import tqdm
# from bs4 import BeautifulSoup
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.downloader.download('vader_lexicon')
# from newspaper import Article

In [2]:
import pandas as pd
import re
from htmldate import find_date
from newspaper import Article
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\minhn\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
def download_news(urls, sentiment_analysis=True):
    if sentiment_analysis:
        articles_df = pd.DataFrame({'publish_date':[], 'title': [],'body_text': [], 'url':[],
                                    'neg':[], 'neu':[], 'pos':[], 'compound':[]})
    else:
        articles_df = pd.DataFrame({'publish_date':[], 'title': [],'body_text': [], 'url':[]})  
                                
    for link in urls:
        article = Article(link)
        article.download()
        
        try:
            article.parse()
            text = article.text

        except: 
            print("I didn't get this")
            continue

        # Initialise sentiment analyser  
        sid = SentimentIntensityAnalyzer()
        # Get positive, negative, neutral and compound scores
        polarity = sid.polarity_scores(text)

        tmpdic = {'publish_date': find_date(link), 'title': article.title, 'body_text': text, 'url': link}

        if sentiment_analysis:
            # Initialise sentiment analyser  
            sid = SentimentIntensityAnalyzer()
            # Get positive, negative, neutral and compound scores
            polarity = sid.polarity_scores(text)
            # Update ticker with the new entry polarity
            tmpdic.update(polarity)
        
        # tmpdic now has all keys and values needed to populate the DataFrame
        articles_df.loc[articles_df.shape[0]] = tmpdic

        return articles_df

In [7]:
def search_for_raw_urls(ticker, page):
    raw_urls = []
    for i in page:
        search_url = f"https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws&start={i}"
        r = requests.get(search_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        atags = soup.find_all('a')
        hrefs = [link['href'] for link in atags]
        raw_urls.extend(hrefs)
    return raw_urls

def clean_urls(urls, exclude_list=['maps', 'policies', 'preferences', 'accounts', 'support'],
                        include_list=["https://finance.yahoo.com/news/"]):
    val = []
    for url in urls:
        if any(exc in url for exc in include_list) and not any(exc in url for exc in exclude_list):
            res = re.findall(r'(https?://\S+html)', url)
            val.append(res)
    return list(set(val))

def get_news_urls(ticker, page, exclude_list=['maps', 'policies', 'preferences', 'accounts', 'support'],
                  include_list=["https://finance.yahoo.com/news/"]):
    raw_urls = search_for_raw_urls(ticker, page)
    return raw_urls
    cleaned_urls = clean_urls(raw_urls, exclude_list, include_list)
    return cleaned_urls

In [10]:
raw_urls = search_for_raw_urls("aapl", page=range(0, 101, 10))
raw_urls

['#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640',
 '#',
 '//www.google.com/policies/terms/',
 '//support.google.com/websearch/answer/86640']

In [11]:
cleaned_urls = clean_urls(raw_urls)
cleaned_urls

[]